<a href="https://colab.research.google.com/github/domenico1995/SM/blob/main/SM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install tensorflow_io

In [82]:
import os
import pandas as pd
import datetime


# directory/folder path
dir_path = r'/content/drive/MyDrive/database_sclere/database_sclere'
dir_path_dataset_indiano =r'/content/drive/MyDrive/database_sclere/database_sclere/Dataset indiano'
dir_path_dataset_italiano =r''

data = pd.DataFrame()

for file_path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, file_path)):
        data = pd.read_csv(os.path.join(dir_path, file_path))

data.columns = ['dati']
data.iloc[0]

dati    1;Caterina Antonacci;F;40;1979;;
Name: ita;T_1;92317;92247;14, dtype: object

In [83]:
dataset_italiano = pd.DataFrame()
dataset_italiano.columns = []
d1=[];d2=[];d3=[];d4=[];d5=[];d6=[];d7=[];d8=[];d9=[]
anno_corrente = datetime.datetime.now().year


In [84]:
for i in range(0, data.size, 1):
  string = data.index[i]
  s =string.split(";")
  if s[0] == "ita":
    d1.append(s[0]); d2.append(s[1]); d3.append(s[2]); d4.append(s[3]) ;d5.append(s[4])
    if len(s) > 5:
      d6.append(s[5]); d7.append(s[6]); d8.append(s[7]);
    else:
      dt =str(data['dati'].iloc[i])
      s1 = dt.split(";")
      d6.append(s1[1]); d7.append(s1[2]); d8.append(s1[3]);

In [85]:
dataset_italiano.insert(loc=0, column='Dataset', value=d1)
dataset_italiano.insert(loc=1, column='Paziente', value=d2)
dataset_italiano.insert(loc=2, column='Foto1', value=d3)
dataset_italiano.insert(loc=3, column='Foto2', value=d4)
dataset_italiano.insert(loc=4, column='Hb', value=d5)
dataset_italiano.insert(loc=5, column='Sesso', value=d7)
dataset_italiano.insert(loc=6, column='Eta', value=d8)
dataset_italiano['Hb'] = dataset_italiano['Hb'].astype(float)
dataset_italiano

,Dataset,Paziente,Foto1,Foto2,Hb,Sesso,Eta
0,ita,T_1,92317,92247,14.0,F,40
1,ita,T_2,93707,93702,13.0,F,34
2,ita,T_3,94015,94012,14.0,M,53
3,ita,T_4,95329,95326,15.0,M,36
4,ita,T_5,74439,74432,13.0,M,51
...,...,...,...,...,...,...,...
86,ita,T_88,82237,82233,16.0,M,41
87,ita,T_89,82828,82831,15.0,M,22
88,ita,T_90,83844,83843,12.0,F,40
89,ita,T_91,85734,85732,15.0,M,33


In [86]:
dataset_indiano = pd.DataFrame()
dataset_indiano.columns = []
d1 = []; d2 = []; d3 = []; d4 = []; d5 = []; d6 = []; d7 = []

In [87]:
for i in range(0, data.size, 1):
  string = data.index[i]
  s =data.index[i].split(";")
  if s[0] == "ind":
    d1.append(s[0]); d2.append(s[1]); d3.append(s[2]); d4.append(s[3]); d5.append(s[4])
    if len(s) > 5:
      d6.append(s[6]);
    else:
      dt =str(data['dati'].iloc[i])
      s1 = dt.split(";")
      d6.append(s1[2]);

In [88]:
dataset_indiano.insert(loc=0, column='Dataset', value=d1)
dataset_indiano.insert(loc=1, column='Paziente', value=d2)
dataset_indiano.insert(loc=2, column='Foto1', value=d3)
dataset_indiano.insert(loc=3, column='Foto2', value=d4)
dataset_indiano.insert(loc=4, column='Hb', value=d5)
dataset_indiano.insert(loc=5, column='Sesso', value=d6 )
dataset_indiano['Hb'] = dataset_indiano['Hb'].astype(float)
dataset_indiano

,Dataset,Paziente,Foto1,Foto2,Hb,Sesso
0,ind,1,20200118_164759,20200118_164733,12.0,M
1,ind,2,20200124_154320,20200124_154319,8.0,F
2,ind,3,20200124_155439,20200124_155418,10.0,F
3,ind,4,20200124_160524,20200124_160522,8.0,F
4,ind,5,20200124_161455,20200124_161452,7.0,F
...,...,...,...,...,...,...
90,ind,91,20200302_231050,20200302_231057,13.0,M
91,ind,92,20200312_183626,20200312_183703,13.0,F
92,ind,93,20200315_161347,20200315_161348,12.0,M
93,ind,94,20200318_130148,20200318_130151,11.0,F


In [89]:
dataset_adu = pd.DataFrame()
dataset_adu.columns = []
d1=[];d2=[];d3=[];d4=[];d5=[];d6=[];d7=[];d8=[];d9=[]

In [90]:
for i in range(0, data.size, 1):
  string = data.index[i]
  s =string.split(";")
  if s[0] == "adu":
    d1.append(s[0]); d2.append(s[1]); d3.append(s[2]); d4.append(s[3]) ;d5.append(s[4])
    if len(s) > 5:
      d6.append(s[5]); d7.append(s[6]); d8.append(s[7]);
    else:
      dt =str(data['dati'].iloc[i])
      s1 = dt.split(";")
      d6.append(s1[1]); d7.append(s1[2]); d8.append(s1[3]);

In [91]:
dataset_adu.insert(loc=0, column='Dataset', value=d1)
dataset_adu.insert(loc=1, column='Paziente', value=d2)
dataset_adu.insert(loc=2, column='Foto1', value=d3)
dataset_adu.insert(loc=3, column='Foto2', value=d4)
dataset_adu.insert(loc=4, column='Hb', value=d5)
dataset_adu.insert(loc=5, column='Sesso', value=d7)
dataset_adu.insert(loc=6, column='Eta', value=d8)
dataset_adu['Hb'] = dataset_adu['Hb'].astype(float)
dataset_adu

,Dataset,Paziente,Foto1,Foto2,Hb,Sesso,Eta
0,adu,1,90755,90756,9.0,F,82
1,adu,2,91409,91408,10.0,F,77
2,adu,3,91622,91559,10.0,F,52
3,adu,4,91745,91743,11.0,F,73
4,adu,5,92209,92152,11.0,F,74
5,adu,6,101040,101031,15.0,F,77
6,adu,7,101258,101257,12.0,F,72
7,adu,8,105831,105828,14.0,M,84
8,adu,9,110508,110455,9.0,M,61
9,adu,10,110956,110931,12.0,M,70


In [92]:
dataset_onc = pd.DataFrame()
dataset_onc.columns = []
d1=[];d2=[];d3=[];d4=[];d5=[];d6=[];d7=[];d8=[];d9=[]

In [93]:
for i in range(0, data.size, 1):
  string = data.index[i]
  s =string.split(";")
  if s[0] == "onc":
    d1.append(s[0]); d2.append(s[1]); d3.append(s[2]); d4.append(s[3]) ;d5.append(s[4])
    if len(s) > 5:
      d6.append(s[5]); d7.append(s[6]); d8.append(s[7]);
    else:
      dt =str(data['dati'].iloc[i])
      s1 = dt.split(";")
      d6.append(s1[1]); d7.append(s1[2]); d8.append(s1[3]);

In [94]:
dataset_onc.insert(loc=0, column='Dataset', value=d1)
dataset_onc.insert(loc=1, column='Paziente', value=d2)
dataset_onc.insert(loc=2, column='Foto1', value=d3)
dataset_onc.insert(loc=3, column='Foto2', value=d4)
dataset_onc.insert(loc=4, column='Hb', value=d5)
dataset_onc.insert(loc=5, column='Sesso', value=d7)
dataset_onc.insert(loc=6, column='Eta', value=d8)
dataset_onc['Hb'] = dataset_onc['Hb'].astype(float)
dataset_onc

,Dataset,Paziente,Foto1,Foto2,Hb,Sesso,Eta
0,onc,O_01C,121623,121621,11.0,F,77
1,onc,O_02C,121832,121812,11.0,F,65
2,onc,O_03C,122134,122128,13.0,M,30
3,onc,O_04C,123021,123018,9.0,F,46
4,onc,O_05C,125520,125519,12.0,M,65
5,onc,O_06C,130005,130004,9.0,M,57
6,onc,O_07C,130235,130232,9.0,M,56
7,onc,O_08C,130432,130430,8.0,M,53
8,onc,O_09C,130814,130807,8.0,M,60
9,onc,O_10C,130959,130957,9.0,M,59


In [95]:
dataset = pd.concat([dataset_italiano, dataset_indiano, dataset_adu, dataset_onc], ignore_index=True)
print(dataset)

    Dataset Paziente   Foto1   Foto2    Hb Sesso Eta
0       ita      T_1   92317   92247  14.0     F  40
1       ita      T_2   93707   93702  13.0     F  34
2       ita      T_3   94015   94012  14.0     M  53
3       ita      T_4   95329   95326  15.0     M  36
4       ita      T_5   74439   74432  13.0     M  51
..      ...      ...     ...     ...   ...   ...  ..
212     onc    O_10C  130959  130957   9.0     M  59
213     onc    O_11C   91410   91408  15.0     M  22
214     onc    O_12C   91505   91502  10.0     F  76
215     onc    O_13C   93333   93332  12.0     F  70
216     onc    O_14C  110116  110115  14.0     F  54

[217 rows x 7 columns]


In [96]:
def definisci_anemia(row):
    if (13.0 <= row['Hb']):
        return 1
    else:
        return 0
def definisci_anemia_lieve(row):
    if (row['Sesso'] == 'M' and 11.0 <= row['Hb'] <= 12.9) or \
       (row['Sesso'] == 'F' and 11.0 <= row['Hb'] <= 11.9):
        return 1
    else:
        return 0
def definisci_anemia_moderata(row):
    if (8.0 <= row['Hb'] <= 10.9):
        return 1
    else:
        return 0
def definisci_anemia_grave(row):
    if (8.0 > row['Hb']):
        return 1
    else:
        return 0
def definisci_sesso(row):
  if (row['Sesso'] == 'F'):
        return 1
  else:
      return 0


In [97]:
dataset['Nessuna'] = dataset.apply(definisci_anemia, axis=1)
dataset['Grave'] = dataset.apply(definisci_anemia_grave, axis=1)
dataset['Moderata'] = dataset.apply(definisci_anemia_moderata, axis=1)
dataset['Lieve'] = dataset.apply(definisci_anemia_lieve, axis=1)
dataset['Sesso'] = dataset.apply(definisci_sesso, axis=1)
dataset.insert(loc=0, column='Indice', value=dataset.index+1)
dataset

,Indice,Dataset,Paziente,Foto1,Foto2,Hb,Sesso,Eta,Nessuna,Grave,Moderata,Lieve
0,1,ita,T_1,92317,92247,14.0,1,40,1,0,0,0
1,2,ita,T_2,93707,93702,13.0,1,34,1,0,0,0
2,3,ita,T_3,94015,94012,14.0,0,53,1,0,0,0
3,4,ita,T_4,95329,95326,15.0,0,36,1,0,0,0
4,5,ita,T_5,74439,74432,13.0,0,51,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
212,213,onc,O_10C,130959,130957,9.0,0,59,0,0,1,0
213,214,onc,O_11C,91410,91408,15.0,0,22,1,0,0,0
214,215,onc,O_12C,91505,91502,10.0,1,76,0,0,1,0
215,216,onc,O_13C,93333,93332,12.0,1,70,0,0,0,0


In [98]:
def elenca_file_cartelle(directory):
    lista_file_cartelle = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            lista_file_cartelle.append(os.path.join(root, file))
        for dir in dirs:
            lista_file_cartelle.append(os.path.join(root, dir))

    return lista_file_cartelle


In [99]:
directory_da_esplorare = r'/content/drive/MyDrive/database_sclere/database_sclere/'

lista = elenca_file_cartelle(directory_da_esplorare)

for elemento in lista:
    print(elemento)

/content/drive/MyDrive/database_sclere/database_sclere/hbvalue.csv
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive
/content/drive/MyDrive/database_sclere/database_sclere/Dataset indiano
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive/Dataset congiuntive gruppo anemia  organizzato 28 mar 2020
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive/Dataset congiuntive italiano segmentato
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive/Dataset congiuntive gruppo anemia  organizzato 28 mar 2020/.DS_Store
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive/Dataset congiuntive gruppo anemia  organizzato 28 mar 2020/Congiuntiva completo riorganizzato 28 mar 2020.xlsx
/content/drive/MyDrive/database_sclere/database_sclere/Italiano congiuntive/Dataset congiuntive gruppo anemia  organizzato 28 mar 2020/Trasfusionale congiuntive
/content/drive/MyDrive/database_sclere/dat